In [1]:
import numpy as np
from numba import njit

## nb_permutations

In [2]:
@njit(error_model="numpy")
def nb_permutations(pattern, length):
    # pattern :1d ndarray, likely np.arange(1,10,2)
    # length :int, length<=pattern.shape[0]
    loop = 1
    n_pattern = pattern.shape[0]
    for i in range(length):
        loop *= n_pattern - i
    return_arr = np.zeros((loop, length), dtype="int8")
    list_index_pattern = np.arange(n_pattern)
    for i in range(loop):
        mask = np.ones(n_pattern, dtype="bool")
        num = i
        for j in range(length):
            index_pattern = (list_index_pattern[mask])[num % (n_pattern - j)]
            return_arr[i, j] = pattern[index_pattern]
            num //= n_pattern - j
            mask[index_pattern] = False
    return return_arr

In [3]:
nb_permutations(np.arange(1,8,2), 2)

array([[1, 3],
       [3, 1],
       [5, 1],
       [7, 1],
       [1, 5],
       [3, 5],
       [5, 3],
       [7, 3],
       [1, 7],
       [3, 7],
       [5, 7],
       [7, 5]], dtype=int8)

## make_dict_change_x_pattern

In [4]:
def make_dict_change_x_pattern(max_ops):
    dict_change_x_pattern = dict()
    dict_change_x_pattern[0] = np.zeros((1, 1), dtype="int8")
    dict_max_loop = np.ones((max_ops + 1, max_ops + 1), dtype="int64")
    for i in range(1, max_ops + 1):
        change_x_pattern = nb_permutations(np.arange(1, max_ops + 2), i)  # not use 0,it is const(1)
        max_drop_num = np.zeros(change_x_pattern.shape[0], dtype="int8")
        for j in range(change_x_pattern.shape[0]):
            max_ = change_x_pattern[j, 0]
            save = max_ - 1
            for k in range(1, change_x_pattern.shape[1]):
                if max_ + 1 == change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                elif max_ + 1 < change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                    save = max_ - 1
            max_drop_num[j] = save
        dict_change_x_pattern[i] = change_x_pattern[np.argsort(max_drop_num)].astype("int8")
        dict_max_loop[i] = np.array([np.sum(max_drop_num <= j) for j in range(np.max(max_drop_num) + 1)])
    return dict_change_x_pattern, dict_max_loop

In [5]:
dict_change_x_pattern, dict_max_loop=make_dict_change_x_pattern(4)
dict_change_x_pattern

{0: array([[0]], dtype=int8),
 1: array([[1],
        [2],
        [3],
        [4],
        [5]], dtype=int8),
 2: array([[1, 2],
        [2, 1],
        [2, 3],
        [3, 1],
        [1, 3],
        [3, 2],
        [3, 4],
        [4, 1],
        [4, 2],
        [4, 5],
        [1, 4],
        [2, 4],
        [4, 3],
        [3, 5],
        [2, 5],
        [5, 2],
        [5, 3],
        [5, 1],
        [1, 5],
        [5, 4]], dtype=int8),
 3: array([[1, 2, 3],
        [2, 1, 3],
        [2, 3, 4],
        [2, 3, 1],
        [1, 3, 4],
        [3, 4, 5],
        [3, 2, 4],
        [3, 4, 1],
        [3, 4, 2],
        [3, 1, 4],
        [3, 2, 1],
        [1, 3, 2],
        [3, 1, 2],
        [4, 1, 3],
        [1, 4, 3],
        [4, 5, 3],
        [4, 2, 3],
        [4, 2, 5],
        [4, 5, 2],
        [1, 4, 5],
        [4, 3, 2],
        [2, 4, 3],
        [1, 2, 4],
        [4, 1, 5],
        [2, 4, 5],
        [4, 5, 1],
        [4, 3, 5],
        [4, 1, 2],
        [4, 3, 1

In [6]:
dict_max_loop

array([[  1,   1,   1,   1,   1],
       [  1,   2,   3,   4,   5],
       [  1,   3,   7,  13,  20],
       [  1,   4,  13,  33,  60],
       [  1,   5,  20,  60, 120]])

In [7]:
n=3
print(f"0 : {dict_change_x_pattern[n][:dict_max_loop[n,0]]}")
for i in range(1,dict_max_loop.shape[0]):
    print(f"{i} : {dict_change_x_pattern[n][dict_max_loop[n,i-1]]}~{dict_change_x_pattern[n][dict_max_loop[n,i]-1]}")

0 : [[1 2 3]]
1 : [2 1 3]~[2 3 1]
2 : [1 3 4]~[3 1 2]
3 : [4 1 3]~[4 2 1]
4 : [1 3 5]~[5 4 3]


## make_change_x_id

In [8]:
def make_change_x_id(mask, x_max):
    # mask -> [3,4,2] みたいな
    # x_max -> 全体としての最大値
    if np.sum(mask) == 0:
        return 0
    TF = np.ones(x_max, dtype="bool")
    return_num = 0
    for i in range(mask.shape[0]):
        return_num *= x_max - i
        return_num += np.sum(TF[: mask[i] - 1])
        TF[mask[i] - 1] = False
    return return_num

## make_dict_mask_x

In [9]:
def make_dict_mask_x(eq_x_max):
    dict_mask_x = dict()
    for i in range(eq_x_max + 1):
        __mask_x = nb_permutations(np.arange(1, i + 1), i)  # 第一要素が0にする。
        mask_x = np.zeros((__mask_x.shape[0], __mask_x.shape[1] + 1), dtype="int8")
        mask_x[:, 1:] = __mask_x  # 第一要素が0にする。
        change_x_num = np.array([make_change_x_id(mask_x[k, 1:], i) for k in range(mask_x.shape[0])])
        index = np.argsort(change_x_num)
        dict_mask_x[i] = mask_x[index].copy()
    return dict_mask_x
make_dict_mask_x(3)

{0: array([[0]], dtype=int8),
 1: array([[0, 1]], dtype=int8),
 2: array([[0, 1, 2],
        [0, 2, 1]], dtype=int8),
 3: array([[0, 1, 2, 3],
        [0, 1, 3, 2],
        [0, 2, 1, 3],
        [0, 2, 3, 1],
        [0, 3, 1, 2],
        [0, 3, 2, 1]], dtype=int8)}

In [10]:
eq_x_max=3
dict_mask_x=make_dict_mask_x(eq_x_max)
for i in range(eq_x_max+1):
    for j in range(dict_mask_x[i].shape[0]):
        print(f"{i}: {make_change_x_id(dict_mask_x[i][j,1:],i)}:{dict_mask_x[i][j]}")

0: 0:[0]
1: 0:[0 1]
2: 0:[0 1 2]
2: 1:[0 2 1]
3: 0:[0 1 2 3]
3: 1:[0 1 3 2]
3: 2:[0 2 1 3]
3: 3:[0 2 3 1]
3: 4:[0 3 1 2]
3: 5:[0 3 2 1]


In [47]:
import numpy as np
from numba import njit
import timeit

@njit(error_model="numpy")
def count_True(arr, mode, border):
    count = 0
    match mode:
        case 0:  # lambda x: x
            for i in arr.flat:
                if i:
                    count += 1
        case 1:  # lambda x: x == border
            for i in arr.flat:
                if i == border:
                    count += 1
        case 2:  # lambda x: x != border
            for i in arr.flat:
                if i != border:
                    count += 1
        case 3:  # lambda x: x < border
            for i in arr.flat:
                if i < border:
                    count += 1
        case 4:  # lambda x: x <= border
            for i in arr.flat:
                if i <= border:
                    count += 1
        case 5:  # lambda x: x > border
            for i in arr.flat:
                if i > border:
                    count += 1
        case 6:  # lambda x: x >= border
            for i in arr.flat:
                if i >= border:
                    count += 1
    return count

@njit(error_model="numpy")
def nb_calc_RPN_1(x, equation):
    int_nan = -100
    n_sample = x.shape[1]
    stack = np.empty((count_True(equation, 6, 0), n_sample), dtype="float64")  # 6 -> lambda x: x >= border
    next_index = 0
    for i in range( equation.shape[0]):
        last_op = equation[i]
        if last_op >= 0:
            for j in range(n_sample):
                stack[next_index,j] = x[last_op,j]
            next_index += 1
        elif last_op == int_nan:
            break
        else:
            calc_arr_binary_op(last_op, stack[next_index - 2], stack[next_index - 1])
            next_index -= 1
    return_stack = np.empty((2, n_sample), dtype="float64")
    _mean = np.mean(stack[0])
    plus_minus_0 = np.sign(stack[0, 0] - _mean)
    plus_minus_1 = np.sign(stack[0, 0])
    sum = 0.
    for i in range(n_sample):
        sum += (stack[0,i] - _mean) ** 2
    std = np.sqrt(sum/n_sample)
    for i in range(n_sample):
        return_stack[0,i] = plus_minus_0 * (stack[0, i] - _mean) / std
        return_stack[1,i] = plus_minus_1 * stack[0, i]
    return return_stack

@njit(error_model="numpy")  # ,fastmath=True)
def set_x(arr_in, arr_out):
    for i in range(arr_in.shape[0]):
        arr_in[i] = arr_out[i]

@njit(error_model="numpy")  # ,fastmath=True)
def calc_arr_binary_op(op, arr1, arr2):
    match op:
        case -1:  # +
            for i in range(arr1.shape[0]):
                arr1[i] += arr2[i]
        case -2:  # -
            for i in range(arr1.shape[0]):
                arr1[i] -=  arr2[i]
        case -3:  # *
            for i in range(arr1.shape[0]):
                arr1[i] *=  arr2[i]
        case -4:  # a / b  (a > b)
            for i in range(arr1.shape[0]):
                arr1[i] /= arr2[i]
        case -5:  # b / a  (a > b)
            for i in range(arr1.shape[0]):
                arr1[i] = arr2[i] / arr1[i]


x = np.random.random((10,300))
#equation = np.array([2,1,5,6,3,-3,-2,-4,-5])
#equation = np.array([2,1,-3,5,-2,6,-4,3,-5])
equation = np.array([2,1,-3,5,-2,6,-4,3,-3])
print(nb_calc_RPN_1(x, equation)[1,0])
loop = 1000000
print(timeit.timeit(lambda: nb_calc_RPN_1(x, equation), number=loop))

0.32940982010606484
1.6831207500072196


In [31]:
(((x[2,0] * x[1,0])-x[5,0]) /x[6,0])*x[3,0]

np.float64(0.9967554393752359)

In [42]:
import array
import numpy as np
from numba import njit
import timeit

@njit(error_model="numpy")
def isclose(a, b, rtol=1e-06, atol=0):
    return np.abs(a - b) <= atol + rtol * np.abs(b)

@njit(error_model="numpy")
def isclose_arr_1(arr1, arr2, rtol=1e-06, atol=0):
    for i in range(arr1.shape[0]):
        if not isclose(arr1[i], arr2[i], rtol=rtol, atol=atol):
            return False
    return True

@njit(error_model="numpy")
def isclose_arr_2(arr1, arr2, rtol=1e-06, atol=0):
    if np.all(isclose(arr1, arr2, rtol=rtol, atol=atol)):
        return True
    return False

arr1 = np.random.random((300))
arr2 = np.random.random((300))

print(isclose_arr_1(arr1, arr2),isclose_arr_2(arr1, arr2))
loop = 1000000
print(timeit.timeit(lambda: isclose_arr_1(arr1, arr2), number=loop))
print(timeit.timeit(lambda: isclose_arr_2(arr1, arr2), number=loop))

False False
0.33317883301060647
0.4938288329867646
